# Modeling Echo Chambers and Polarization Dynamics in Social Networks | [Baumann et al. (2020)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.124.048301)

A simple model of opinion dynamics that can reproduce two empirical features frequently observed in polarized social networks: 
1. more-active users, strongly engaged in social interactions, tend to show more-extreme opinions
1. the opinion expressed by a user and those expressed by their neighbors in the social interaction network are similar. 

The model introduces a mechanism by which agents sharing similar opinions can mutually reinforce each other and move toward more-extreme views, thus describing a *radicalization dynamics*, also known as group polarization in social psychology.

Alongside, opinion states are coupled to the underlying time-varying network of social interactions by *homophily*.

## Social System 

$$\mathcal{S} = (\mathcal{T}_S, \mathcal{\Theta}_S, \mathcal{G}_S, \Pi_S)$$

### State Space 

$$\mathcal{G}_S = (V, E(t))$$

The contact pattern among agents, sustaining the opinion dynamics, represents social interactions encoded in an homophily-based activity-driven (AD) temporal directed graph $\mathcal{G}_S$ represented by $A_{ij}(t)$.

### Parameter Space 

$$\mathcal{\Theta}_S = (\epsilon,\gamma,m,\beta)$$

* $\epsilon$: minimum activity 
* $\gamma$: power law exponent
* $m$: contacts
* $\beta$: homophily controlling power law decay exponent of the connection probability $p_{ij}$ with opinion distance (may include various endogenous and/or exogenous homophilic biases)

### Dynamics

$$\Pi_S : \mathcal{T}_S \times \mathcal{G}_S \to [0,1]\subset\mathbb{R}$$

The probability that an active agent $i$ will contact $j$ is modeled as a decreasing function of the distance between their opinions

$$\pi_{ij}(t)=\frac{|x_i(t)-x_j(t)|^{-\beta}}{\sum_j|x_i(t)-x_j(t)|^{-\beta}}.$$


## Opinion System

$$\mathcal{O} = (\mathcal{T}_O, \mathcal{\Theta}_O, \mathcal{\Gamma}_O, \Phi_O)$$

### State Space 

$$\mathcal{\Gamma}_O = \mathbb{R}^N$$

* $x_i(t) \in \mathbb{R}$: one-dimensional opinion of agent $i$
* $\sigma(x_i)=sign(x_i)$: qualitative stance of $i$ towards a binary issue of choice 
* $|x_i|$: quantitative stance / strength of $i$'s opinion (the larger, the more extreme)

### Parameter Space 

$$\mathcal{\Theta}_O = (N,K,\alpha,r,a)$$

* $N$: population size
* $K$: agent-agent social interaction strength 
* $\alpha$: *controversialness* whcih tunes the degree of nonlinearity between an agent’s opinion and the sigmoidal social influence they exert on others 
* $r$: reciprocity rate / probability of directed interactions (when $i$ establishes a connection to $j$, $j$ will update its opinion, but $i$ will do the same only if the interaction is reciprocated)
* $a_i \in [\epsilon, 1]$: activity of $i$ representing their propensity to contact $m$ distinct random other agents.

### Dynamics

The opinion dynamics is a purely collective, self-organized process which is exclusively driven by the interactions among agents such that the opinion of $i$ changes depending on the aggregated inputs from its neighbors. we model opinion dynamics as a purely collective, self-organized process without any intrin- sic individual preferences.

$$\dot{x_i}=-x_i + K\sum_{j=1}^NA_{ij}\tanh(\alpha x_j)$$

The neighbors of $i$ are determined by the temporal adjacency matrix $A_{ij}(t)$ such that 
* $A_{ij}(t) = 1$ if there is input from $j$ to $i$ at time $t$
* $A_{ij}(t) = 0$ otherwise.

#### Radicalization 

If $\sigma(x_i) = \sigma(x_j)$, the interaction will cause an increase of both convictions and hence reinforce opinions.

#### Convergence

If $\sigma(x_i) = -\sigma(x_j)$, opinions tend to converge. 

#### Neutrality 

The opinions of agents lacking social interactions decay toward the neutral state.

## Relevant Regime 

$$|\mathcal{T}_O| \ll |\mathcal{T}_S|$$

The analysis is focused on a regime in which social interactions evolve much faster than opinions, like it is reasonable to assume for online social media. This yields a clear timescale separation between the social and opinion dynamics.

## Implementation 

### Modules

In [18]:
# Benchmarking
using BenchmarkTools

# Data Management 
using DataFrames, DataFramesMeta, DrWatson, Queryverse;

# Data Visualization
using GraphPlot, Plots, AgentsPlots, PlotThemes;

# Statistics
using Random, Distributions, StatsBase;

# Graph Modelling
using LightGraphs, SimpleWeightedGraphs, GraphIO;

# Dynamical Systems Modelling
using Agents ###DynamicalSystems, DifferentialEquations, StochasticDiffEq;

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %%]  20.7 %                         ]  35.2 %Fetching: [===============>                         ]  35.3 %>                    ]  48.3 %>                    ]  48.8 %=========================>               ]  61.9 %]  74.9 %====================================>    ]  87.9 %====================================>    ]  89.9 %

  Resolving package versions...
  Installed BenchmarkTools ─ v0.5.0
   Updating `~/.julia/environments/v1.4/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
  [6e34b625] ↑ Bzip2_jll v1.0.6+2 ⇒ v1.0.6+3
  [d7e528f0] ↑ FreeType2_jll v2.10.1+2 ⇒ v2.10.1+3
  [559328eb] ↑ FriBidi_jll v1.0.5+3 ⇒ v1.0.5+4
  [c1c5ebd0] ↑ LAME_jll v3.100.0+1 ⇒ v3.100.0+2
  [e7412a2a] ↑ Ogg_jll v1.3.4+0 ⇒ v1.3.4+1
  [458c3c95] ↑ OpenSSL_jll v1.1.1+4 ⇒ v1.1.1+5
  [91d4177d] ↑ Opus_jll v1.3.1+1 ⇒ v1.3.1+2
  [83775a58] ↑ Zlib_jll v1.2.11+14 ⇒ v1.2.11+15
  [0ac62f75] ↑ libass_jll v0.14.0+2 ⇒ v0.14.0+3
  [f638f0a6] ↑ libfdk_aac_jll v0.1.6+2 ⇒ v0.1.6+3
  [f27f6e37] ↑ libvorbis_jll v1.3.6+4 ⇒ v1.3.6+5
  [dfaa095f] ↑ x265_jll v3.0.0+1 ⇒ v3.0.0+2
┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


### Agent Type 

In [2]:
# Voter Definition
mutable struct User <: AbstractAgent
    id::Int              # arbitrarily initialized identifier
    pos::Int             # arbitrarily initialized position
    opinion::Real        # extracted from Uniform([-1,1])
    activity::Real       # extracted from Power([ϵ,1],γ)
end

### Functions 

In [3]:
# Power law sampling
function powelaw_sample(a::Real, b::Real, γ::Real)
    a + (b-a)*rand()^γ
end;

# Connection rate 
function connection_rate(agent, friend, β)
    return abs(agent.opinion -friend.opinion)^(-β)
end;

### Parameters

#### Social

In [4]:
ϵ = 0.1
γ = 2.1
β = 2
m = 10

# Initialize number of steps
social_time_steps = 100;

#### Opinion

In [5]:
# Set the user population size
N = 100
K = 3
α = 0.05
r = 0.5

# Initialize number of steps
opinion_time_steps = 10;

### Model 

In [6]:
# Model initialization 
function initialize_model(N::Int, K::Real, α::Real, r::Real,
                    ϵ::Real, γ::Real, β::Real, m::Int)
    # Define the dictionary of model properties
    properties = @dict(N, K, α, r,
                       ϵ, γ, β, m)
    
    # Create the ambient space
    space = GraphSpace(SimpleWeightedDiGraph(N)) 
    
    # Instantiate the model
    model = ABM(User, space; properties = properties)
    
    # Populate the model
    for id ∈ 1:N
        pos = id
        opinion = rand(Uniform(-1, 1))
        activity = powelaw_sample(ϵ, 1, γ)
        add_agent!(pos, model, opinion, activity)
    end
    
    return model
end;

# Activation 
function activate!(agent, model)
    others = [a for a ∈ allagents(model) if a != agent]
    deltas = [abs(agent.opinion-other.opinion)^(-model.properties[:β]) for other ∈ others]
    friends = []
    friend = rand(others)
    while length(friends) ≤ model.properties[:m] || friend ∈ friends
        friend = rand(others)
        if rand() ≤ connection_rate(agent, friend, model.properties[:β]) / sum(deltas)
            push!(friends, friend)
        end
    end
    return friends
end

# Interaction 
function follow!(agent, model, friends)
    for friend ∈ friends
        add_edge!(model.space.graph, agent.pos, friend.pos, model.properties[:K])
    end
end;

function reciprocate!(agent, model)
    for inneighbor ∈ node_neighbors(agent, model; neighbor_type=:in)
        follower = get_node_agents(inneighbor, model)[1]
        if rand() ≤ model.properties[:r]
            add_edge!(model.space.graph, agent.pos, follower.pos, model.properties[:K])
        end
    end
end

# Update
function update!(agent, model)
    for outneighbor ∈ node_neighbors(agent, model; neighbor_type=:out)
        friend = get_node_agents(outneighbor, model)[1]
        agent.opinion += LightGraphs.weights(model.space.graph)[agent.pos, friend.pos] * tanh(model.properties[:α]*friend.opinion)
    end
end;

# Individial dynamics
function step!(agent, model)
    if rand() ≤ agent.activity
        friends = activate!(agent, model)
        follow!(agent, model, friends)
        reciprocate!(agent, model)
        update!(agent, model)
    end
end;

### Simulation

In [20]:
# Seed Selection
Random.seed!(1234);

# Model Instantiation 
model = initialize_model(N, K, α, r,
                         ϵ, γ, β, m)
print(model)

# Data Collection
data = @time run!(model, step!, 100);

sort!(DataFrame(allagents(model)), :opinion, rev = false)

AgentBasedModel with 100 agents of type User
 space: GraphSpace with 100 nodes and 0 edges
 scheduler: fastest
 properties: Dict{Symbol,Real}(:α => 0.05,:ϵ => 0.1,:N => 100,:γ => 2.1,:m => 10,:K => 3,:β => 2,:r => 0.5)  2.334339 seconds (28.32 M allocations: 488.207 MiB, 2.25% gc time)


,id,pos,opinion,activity
,Int64,Int64,Float64,Float64
1,7,7,-5140.6,0.920205
2,47,47,-4894.65,0.638432
3,54,54,-4818.67,0.74146
4,20,20,-4792.99,0.78008
5,45,45,-4344.62,0.7153
6,10,10,-4294.29,0.728305
7,73,73,-4115.1,0.520851
8,36,36,-4041.14,0.877997
9,89,89,-3874.09,0.507113
